In [ ]:
#default_exp controls

# Controls

> In this module, I implement the physical controls for `DecadesPlayer`.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
from nbdev.showdoc import *
from IPython.display import clear_output

In [ ]:
#export
import time
import smbus
from time_travel_radio.core import *
import numpy as np
from gpiozero import Button
import os
from typing import *

## Abstract Classes

First we must define some abstract classes so we can use our potentiometer and Analog-Digital Converter (ADC). Largely borrowed from [Freenove ADC Module](https://github.com/Freenove/Freenove_Ultimate_Starter_Kit_for_Raspberry_Pi/blob/master/Code/Python_Code/07.1.1_ADC/ADC.py).

In [ ]:
#export
class ADCDevice:
    """ADC device that automatically closes itself when destroyed."""
    def __init__(self):
        self.cmd = 0
        self.address = 0
        self.bus=smbus.SMBus(1)
            
    def close(self):    self.bus.close()
    def __del__(self):  self.close()
    def __exit__(self): self.close()

In [ ]:
#export
class ADS7830(ADCDevice):
    def __init__(self):
        super().__init__()
        self.cmd = 0x84
        self.address = 0x4b # 0x4b is the default i2c address for ADS7830 Module.   
        
    def analogRead(self, chn): # ADS7830 has 8 ADC input pins, chn:0,1,2,3,4,5,6,7
        value = self.bus.read_byte_data(self.address, self.cmd|(((chn<<2 | chn>>1)&0x07)<<4))
        return value
    
    @staticmethod
    def value_range(): 
        """Returns range of values device can represent."""
        return 2**8 - 1  # This is an 8-bit ADC

## Controlling MusicPlayer With Potentiometer

In [ ]:
#export
class DecadesDial(ADS7830):
    """Potentiometer linked to ADS7830 ADC converts read value into decade."""
    def __init__(self, decades: List[str]):
        super().__init__()
        self.decades = decades
        self.bins = self._build_bins(len(decades))
    
    def read_decade(self):
        """Reads value from potentiometer and returns the corresponding decade."""
        value = self.analogRead(0)  # B/c potentiometer is connected to channel 0 of ADC
        bin_idx = np.digitize(value, self.bins)
        return self.decades[bin_idx]
    
    def _build_bins(self, num_bins: int):
        bin_size = self.value_range() / num_bins
        bins = [bin_size * i for i in range(1, num_bins)]
        return bins

Let's test it out now. Make sure to first begin playing music on the Raspberry Pi through the normal Spotify client, or else the `DecadesPlayer` won't be able to find the Raspberry Pi.

In [ ]:
player = DecadesPlayer("Brandon's Raspberry Pi")

In [ ]:
dial = DecadesDial([dec for dec in player.playlists])
old_decade = dial.read_decade()
try:
    while True:
        new_decade = dial.read_decade()
        if new_decade != old_decade:
            print(f"Playing the {new_decade}")
            player.play_music(new_decade)
            old_decade = new_decade
        time.sleep(0.01)
except KeyboardInterrupt:
    pass

## Power Button

Next we'll implement a power button that'll allow us to shutdown the Raspberry Pi safely before unplugging it.

Luckily, `gpiozero` has a wonderful `Button` class.

In [ ]:
#export
class PowerButton(Button):
    """Creates a power button for the Raspberry Pi.
    
    Shuts down the Pi after being held down for 5
    seconds.
    """
    def __init__(self, pin):
        """Creates power button from button connected to pin."""
        super().__init__(pin, hold_time=5)
    
    @staticmethod
    def when_held():
        """Shuts down Raspberry Pi."""
        os.system("sudo shutdown -h now")

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted 01_controls.ipynb.
Converted index.ipynb.
